In [61]:
from lunarLanding import DQNAgent

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Dropout, Activation
from keras.optimizers import Adam
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())

import gym

import numpy as np
from scipy.special import softmax
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
import time

import os

['/job:localhost/replica:0/task:0/device:GPU:0']


In [62]:
env = gym.make('LunarLander-v2')
state_size = 8
action_size = 4
agent = DQNAgent(state_size, action_size)

In [63]:
m = Sequential()
m.add(Dense(100, input_dim = 8))
m.add(Activation('elu'))

m.add(Dense(50))
m.add(Activation('elu'))

m.add(Dense(32))
m.add(Activation('elu'))

m.add(Dense(4))
m.add(Activation('softmax'))

m.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.0001))
m.summary()

agent.model = m

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 100)               900       
_________________________________________________________________
activation_61 (Activation)   (None, 100)               0         
_________________________________________________________________
dense_62 (Dense)             (None, 50)                5050      
_________________________________________________________________
activation_62 (Activation)   (None, 50)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 32)                1632      
_________________________________________________________________
activation_63 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_64 (Dense)             (None, 4)                 132       
__________

In [ ]:
done = False
batch_size = 30
game_history = [0]


explore = True
t_steps = 0
for episode in range(300):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
#     print(state.shape)
    total_reward = 0
    prev_reward = 0
    start = time.time()
    for timee in range(450):
        # env.render()
#         if time % 5 == 0:
#             print(time, end=', ')
        

#         print(state)
        action = agent.act(state)
        
        next_state, reward, done, _ = env.step(action)
        

        next_state = np.reshape(next_state, [1, state_size])

        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            break
        
        prev_reward = reward
        total_reward += reward
        if len(agent.memory) > 2000:
            agent.replay(batch_size)
            explore = False
#         print("diff:", reward - prev_reward,"prev:", prev_reward, " current:", reward, "total reward:", total_reward)


    game_history.append(total_reward)
#     if explore == False:
    if len(game_history) > 35:
        print("epi:{} score: {} mean:{} spent:{}".format(episode, total_reward, np.mean(game_history[-30:-1]), time.time() - start))
    else:
        print("epi:{} score: {}spent:{}".format(episode, total_reward, time.time() - start))
        
    if episode % 10 == 9:
        agent.save_model()
        np.save("game_history", game_history)

    done = False

epi:0 score: -28.62593769787399spent:0.013043403625488281
epi:1 score: 14.684772251046905spent:0.009779214859008789
epi:2 score: -273.9855349451718spent:0.011420726776123047
epi:3 score: -158.03376120105926spent:0.016477584838867188
epi:4 score: -177.9290876588574spent:0.01842975616455078
epi:5 score: -14.389761449934111spent:0.015700578689575195
epi:6 score: -119.08245512627458spent:0.019420385360717773
epi:7 score: -209.67786894992963spent:0.019994258880615234
epi:8 score: 81.16347448353785spent:0.013658285140991211
epi:9 score: -93.58004419880008spent:0.012328147888183594
epi:10 score: -177.08894294511433spent:0.01501774787902832
epi:11 score: -240.27343709115317spent:0.013387441635131836
epi:12 score: -3.5259187707101987spent:0.018529176712036133
epi:13 score: -41.368709157105684spent:0.01035308837890625
epi:14 score: -257.80402031507543spent:0.01998591423034668
epi:15 score: -22.6442735832694spent:0.01681351661682129
epi:16 score: 22.524781687165607spent:0.01569962501525879
epi:17

In [28]:
from sklearn.metrics import log_loss

In [33]:
log_loss([0,0,0,0.8],[0.9,0.05,0.02,0.03])

ValueError: Unknown label type: ([0, 0, 0, 0.8],)